In [4]:
# Value Creation Decomposition Toolkit (aligned to Achleitner et al. 2010; Achleitner, Braun & Puche 2015; Söffge & Braun 2017)

import pandas as pd
from pathlib import Path

# Read the CSV file and parse dates
def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        candidate = (parent / rel_path)
        if candidate.exists():
            return candidate.resolve()
    # Helpful diagnostics
    raise FileNotFoundError(
        f"Couldn't locate '{rel_path.as_posix()}' from {here} by walking up {max_up} levels.\n"
        f"- Current working directory: {here}\n"
        f"- Checked: {[str((p / rel_path)) for p in [here, *here.parents][: max_up + 1]]}"
    )

TARGET_CSV = (find_upwards(Path("ValueCreation")) / "Data" / "working.csv")

# Load
df = pd.read_csv(TARGET_CSV)
df['reference_date'] = pd.to_datetime(df['reference_date'])

# Compute row-level ratios and placeholders
df['ebitda_margin']    = df['ebitda'] / df['revenue']
df['ev_to_ebitda']    = df['enterprise_value'] / df['ebitda']
df['de_ratio']        = df['net_debt'] / df['equity']
df['dividends']       = 0.01 * df['equity']
df['capex_injection'] = 0.01 * df['equity']

interest_rate = 0.05

grouped = df.groupby('deal_id')

for deal_id, group in grouped:
    group = group.sort_values('reference_date')
    entry_date = group.iloc[0]['reference_date']
    exit_date  = group.iloc[1]['reference_date']
    # Compute holding period in years
    hold_days  = (exit_date - entry_date).days
    hold_years = hold_days / 365.0

    # Average debt/equity across entry and exit
    avg_de = group['de_ratio'].mean()

    # Cost of debt factor (compounded at 5%)
    cost_debt = (1 + interest_rate) ** hold_years

    # Assign back to the DataFrame (on both rows of the deal)
    df.loc[group.index, 'holding_period_years']  = hold_years
    df.loc[group.index, 'avg_de_ratio']          = avg_de
    df.loc[group.index, 'cost_of_debt_factor']   = cost_debt


